# 1. Load Dataset 🔄

In [82]:
import os
import numpy as np
import random
import csv
from typing import List, Tuple, Any
import math
import pickle

In [83]:
def load_dataset(file_path: str) -> Tuple[List[List[float]], List[Any]]:
    """
    Loads the dataset from a CSV file.

    This function assumes that the CSV file has a header row and that:
    - All columns except the last one are features (converted to float).
    - The last column is the label (left as a string; convert if needed).

    Note:
      The current implementation is particularly suited for datasets like the Iris dataset.
      For other datasets, you might want to modify the logic (e.g., to change the label column index).

    Parameters:
      file_path (str): Path to the dataset file.

    Returns:
      Tuple[List[List[float]], List[Any]]:
          - data: A list of rows, each row is a list of features as floats.
          - labels: A list of labels corresponding to each row.
    """
    data: List[List[float]] = []
    labels: List[Any] = []

    try:
        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            header = next(reader, None)  # Skip header row if exists

            # Process each row in the CSV file
            for row in reader:
                if row:  # Ensure the row is not empty
                    # Convert all columns except the last one into floats (features)
                    try:
                        features = [float(item) for item in row[:-1]]
                    except ValueError as ve:
                        print(f"Could not convert features to float in row: {row}. Error: {ve}")
                        continue
                    # The last column is considered the label (remains as string or processed further)
                    label = row[-1]

                    data.append(features)
                    labels.append(label)

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")

    return data, labels

In [84]:
    dataset, labels = load_dataset('/content/drive/MyDrive/Colab Notebooks/Iris Classification/Iris.csv')
    print(dataset[:3],"\n")
    print(labels[:3])

[[1.0, 5.1, 3.5, 1.4, 0.2], [2.0, 4.9, 3.0, 1.4, 0.2], [3.0, 4.7, 3.2, 1.3, 0.2]] 

['Iris-setosa', 'Iris-setosa', 'Iris-setosa']


In [85]:
def validate_dataset(dataset: List[List[Any]]) -> bool:
    """
    بررسی صحت ساختار و یکپارچگی مجموعه داده.

    این تابع بررسی می‌کند که:
      - مجموعه داده خالی نباشد.
      - تمامی ردیف‌های مجموعه داده دارای تعداد ویژگی یکسان باشند.

    پارامترها:
      dataset (List[List[Any]]): لیستی از ردیف‌های داده؛ هر ردیف نیز لیستی از ویژگی‌هاست.

    خروجی:
      bool: اگر مجموعه داده صحیح باشد، مقدار True و در غیر این صورت False برمی‌گرداند.
    """
    # اگر مجموعه داده خالی باشد، معتبر نیست.
    if not dataset:
        return False

    # تعداد ویژگی‌ها در اولین ردیف را به عنوان مبنا در نظر می‌گیریم.
    num_features = len(dataset[0])
    for row in dataset:
        # بررسی می‌کند که هر ردیف همان تعداد ویژگی با ردیف اولیه داشته باشد.
        if len(row) != num_features:
            return False
    return True

In [86]:
    # نمونه‌ای از یک مجموعه داده معتبر (مثلاً برای دیتاست Iris یا سایر دیتاست‌ها)
    valid_dataset = [
        [5.1, 3.5, 1.4, 0.2],
        [4.9, 3.0, 1.4, 0.2],
        [6.2, 3.4, 5.4, 2.3]
    ]

    # نمونه‌ای از یک مجموعه داده نامعتبر (یک ردیف با تعداد ویژگی متفاوت)
    invalid_dataset = [
        [5.1, 3.5, 1.4, 0.2],
        [4.9, 3.0, 1.4],  # این ردیف تعداد ویژگی کمتری نسبت به بقیه دارد.
        [6.2, 3.4, 5.4, 2.3]
    ]

    # اجرای تابع و چاپ نتایج
    print("مجموعه داده معتبر:", validate_dataset(valid_dataset))    # خروجی: True
    print("مجموعه داده نامعتبر:", validate_dataset(invalid_dataset))  # خروجی: False

مجموعه داده معتبر: True
مجموعه داده نامعتبر: False


In [87]:
def test_data_loading(file_path: str) -> None:
    """
    آزمون عملکرد تابع بارگذاری داده.

    ورودی:
      file_path (str): مسیر فایل CSV برای بارگذاری داده‌ها.

    خروجی:
      None: نتیجه تست از طریق چاپ پیام به کنسول نمایش داده می‌شود.

    توضیحات:
      این تابع ابتدا داده‌ها و برچسب‌ها را از فایل ورودی با استفاده از load_dataset می‌خواند.
      سپس با استفاده از validate_dataset اعتبار داده‌ها را بررسی می‌کند.
      در صورت موفقیت، پیام موفقیت را چاپ می‌کند و در صورت بروز خطا، پیغام مناسب را چاپ می‌کند.
    """
    try:
        dataset, labels = load_dataset(file_path)
        assert validate_dataset(dataset), "Dataset validation failed."
        print("Data loaded and validated successfully.")
    except AssertionError as error:
        print(f"Test failed: {error}")
    except FileNotFoundError:
        print("File not found. Make sure the dataset file exists at the specified path.")
    except Exception as error:
        print(f"An error occurred: {error}")

In [88]:
test_data_loading('/content/drive/MyDrive/Colab Notebooks/Iris Classification/Iris.csv')

Data loaded and validated successfully.


# 2. Normalize the Data 🔄

In [89]:
def transpose(data: List[List]) -> List[List]:
    """
    ترانهاده کردن یک لیست دوبعدی.

    پارامترها:
        data (List[List]): داده‌ها به‌صورت لیست از لیست‌ها (مثلاً ماتریس)

    خروجی:
        List[List]: داده‌های ترانهاده‌شده
    """
    # علامت ستاره داده ها را آنپک میکند
    # تابع زیپ ستون n از هر سطر را کنار هم قرار میدهد و بدین وسیله داده ها را ترانسپوزه میکند.
    return [list(row) for row in zip(*data)]

In [90]:
def min_max_normalizer(data: List[List[float]]) -> List[List[float]]:
    """
    این تابع داده‌های عددی رو با استفاده از روش Min-Max نرمال می‌کنه.
    یعنی همه‌ی اعداد رو بین 0 و 1 میاره، تا مقایسه و آموزش مدل ساده‌تر بشه.
    """

    # چرخوندن ماتریس برای بررسی هر ویژگی به‌صورت ستونی
    transposed_data = transpose(data)

    # پیدا کردن مینیمم و ماکزیمم هر ستون
    min_vals = [min(col) for col in transposed_data]
    max_vals = [max(col) for col in transposed_data]

    scaled_data = []

    # نرمال کردن هر مقدار با استفاده از فرمول min-max
    for row_index, row in enumerate(data):
        scaled_row = []
        for val, min_val, max_val in zip(row, min_vals, max_vals):
            range_val = max_val - min_val
            if range_val == 0:
                scaled_row.append(0.0)  # اگر همه مقادیر برابر بودن، خروجی رو صفر بزار
            else:
                scaled_val = (val - min_val) / range_val
                scaled_row.append(scaled_val)
        scaled_data.append(scaled_row)

    return scaled_data

In [107]:
def split_data(dataset: List[List[float]], training_size: float = 0.7, validation_size: float = 0.0) -> Tuple[List[List[float]], List[List[float]], List[List[float]]]:
    """
    این تابع دیتاست رو به سه بخش تقسیم می‌کنه: آموزش، اعتبارسنجی و تست.

    پارامترها:
    dataset (list of lists): دیتای ورودی که می‌خوایم تقسیمش کنیم.
    training_size (float): درصدی از دیتا که برای آموزش استفاده میشه (مثلاً ۰.۷ یعنی ۷۰٪).
    validation_size (float): درصدی از دیتا که برای اعتبارسنجی استفاده میشه (مثلاً ۰.۱۵ یعنی ۱۵٪).

    خروجی:
    یه تاپل شامل سه تا لیسته: دیتای آموزش، دیتای اعتبارسنجی، دیتای تست.
    """

    # اول دیتا رو به صورت تصادفی قاطی می‌کنیم که ترتیبش تأثیر نذاره
    random.shuffle(dataset)

    # اندازه کل دیتا
    total_size = len(dataset)

    # محاسبه مرز بین بخش‌ها
    train_end = int(training_size * total_size)
    val_end = int((training_size) * total_size)

    # برش دادن دیتا بر اساس درصدها
    training_data = dataset[:train_end]
    # validation_data = dataset[train_end:val_end]
    validation_data = [[]]
    test_data = dataset[val_end:]

    return training_data, validation_data, test_data


In [92]:
data = [
    [5.1, 3.5, 1.4, 0.2],
    [4.9, 3.0, 1.4, 0.2],
    [6.2, 3.4, 5.4, 2.3],
    [5.9, 3.0, 5.1, 1.8],
    [5.4, 3.9, 1.7, 0.4],
    [6.7, 3.1, 4.7, 1.5],
    [5.6, 2.8, 4.9, 2.0],
    [5.7, 2.1, 4.0, 2.1],
    [5.8, 2.2, 9.4, 1.2],
    [5.9, 2.3, 5.5, 0.8],
]

train, val, test = split_data(data, training_size=0.5, validation_size=0.2)
print("Train:", train)
print("Validation:", val)
print("Test:", test)


Train: [[6.2, 3.4, 5.4, 2.3], [5.9, 2.3, 5.5, 0.8], [5.6, 2.8, 4.9, 2.0], [5.9, 3.0, 5.1, 1.8], [5.4, 3.9, 1.7, 0.4]]
Validation: [[5.7, 2.1, 4.0, 2.1], [5.1, 3.5, 1.4, 0.2]]
Test: [[6.7, 3.1, 4.7, 1.5], [5.8, 2.2, 9.4, 1.2], [4.9, 3.0, 1.4, 0.2]]


In [93]:
def test_split_data():
    """
    این تابع بررسی می‌کنه که تابع split_data درست کار می‌کنه یا نه.
    توی تست، اول دیتاست رو لود می‌کنیم، بعد نرمالایزش می‌کنیم، بعد تقسیمش می‌کنیم.
    بعدش چک می‌کنیم که هر بخش از دیتا خالی نباشه و مجموعشون هم برابر با دیتای اولیه باشه.
    """
    try:
        # مسیر فایل دیتاست
        file_path = '/content/drive/MyDrive/Colab Notebooks/Iris Classification/Iris.csv'

        # لود کردن دیتا و لیبل‌ها
        dataset, labels = load_dataset(file_path)

        # نرمال‌سازی دیتا با Min-Max
        dataset = min_max_normalizer(dataset)

        # تقسیم دیتا به سه بخش
        training_data, validation_data, test_data = split_data(dataset)

        # تست اینکه هیچ‌کدوم از بخش‌ها خالی نباشه
        assert len(training_data) > 0, "داده‌های آموزش خالیه."
        assert len(validation_data) > 0, "داده‌های اعتبارسنجی خالیه."
        assert len(test_data) > 0, "داده‌های تست خالیه."

        # تست اینکه تعداد کل داده‌ها تغییر نکرده باشه
        total_size = len(training_data) + len(validation_data) + len(test_data)
        assert total_size == len(dataset), "خطا در تقسیم دیتا: تعداد نهایی با اولیه نمی‌خونه."

        print("✅ تست تقسیم دیتا با موفقیت انجام شد.")

    except AssertionError as error:
        print(f"❌ تست شکست خورد: {error}")

    except Exception as error:
        print(f"⚠️ یه خطایی پیش اومد: {error}")


In [94]:
test_split_data()

✅ تست تقسیم دیتا با موفقیت انجام شد.


# 3. Define the Architecture 🏗

In [95]:
class Neuron:
    """کلاس یک نورون ساده که برای محاسبه خروجی، گرادیان و به‌روزرسانی وزن‌ها استفاده می‌شود."""

    def __init__(self, weights: List[float], bias: float = None):
        """
        نورون با وزن‌ها و بایاس داده‌شده مقداردهی اولیه می‌شود.

        آرگومان‌ها:
        weights: لیستی از وزن‌ها برای ورودی‌های نورون.
        bias: مقدار بایاس برای نورون. اگر داده نشود، یک مقدار تصادفی بین -0.1 و 0.1 به آن اختصاص می‌یابد.
        """
        self.weights = weights
        self.bias = bias if bias is not None else random.uniform(-0.1, 0.1)
        self.inputs = []  # ورودی‌های نورون را ذخیره می‌کند
        self.output = 0   # خروجی نورون را ذخیره می‌کند

    def forward(self, inputs: List[float]) -> float:
        """
        خروجی نورون را با استفاده از ورودی‌ها و وزن‌ها محاسبه می‌کند.

        آرگومان‌ها:
        inputs: لیستی از ورودی‌ها به نورون.

        برمی‌گرداند:
        خروجی نورون.
        """
        self.inputs = inputs  # ذخیره ورودی‌ها برای استفاده در مراحل بعدی
        weighted_sum = sum([input_ * weight for input_, weight in zip(inputs, self.weights)])
        self.output = weighted_sum + self.bias  # جمع کردن وزن‌ها و بایاس
        return self.output

    # def activation(self, output: float) -> float:
    #     """
    #     تابع فعال‌سازی را بر روی خروجی نورون اعمال می‌کند.

    #     در اینجا، به طور ساده از تابع سیگموید استفاده می‌کنیم.
    #     """

    #     # از تابع فعال‌سازی سیگموید استفاده می‌کنیم
    #     return 1 / (1 + (2.718 ** -output))  # این یعنی سیگموید

    def activation(self, output: float) -> float:
        return max(0, output)

    def compute_gradient(self, delta: float) -> List[float]:
        """
        گرادیان برای وزن‌ها را با استفاده از دلتا (سیگنال خطای لایه بعدی) محاسبه می‌کند.

        آرگومان‌ها:
        delta: سیگنال خطا از لایه بعدی.

        برمی‌گرداند:
        لیستی از گرادیان‌ها برای هر وزن.
        """
        gradients = [delta * input_ for input_ in self.inputs]  # محاسبه گرادیان‌ها
        return gradients

    def update_weights(self, learning_rate: float, gradients: List[float]):
        """
        وزن‌ها و بایاس نورون را با استفاده از گرادیان‌ها و نرخ یادگیری به‌روزرسانی می‌کند.

        آرگومان‌ها:
        learning_rate: نرخ یادگیری برای به‌روزرسانی وزن‌ها.
        gradients: گرادیان‌های محاسبه‌شده برای هر وزن.
        """
        # به‌روزرسانی وزن‌ها
        self.weights = [w - learning_rate * g for w, g in zip(self.weights, gradients)]
        self.bias -= learning_rate * gradients[-1]  # بایاس را نیز به‌روزرسانی می‌کنیم

    def propagate_error_back(self) -> List[float]:
        """
        این متد خطای نورون رو برای لایه قبلی حساب می‌کنه.
        یعنی اول مشتق سیگموید رو از خروجی خودش می‌گیره،
        بعد با هر وزن ضرب می‌کنه تا بگه هر ورودی چقدر در خطا سهم داره.
        """
        # مشتق سیگموید: output * (1 - output)
        deriv = self.output * (1 - self.output)
        # برای هر وزن، سهم خطا = مشتق * وزن
        return [deriv * w for w in self.weights]

In [96]:
# تست عملکرد کلاس نورون

# ایجاد یک نورون با وزن‌ها و بایاس تصادفی
weights = [random.uniform(-1, 1) for _ in range(3)]  # سه ورودی با وزن‌های تصادفی
bias = random.uniform(-0.1, 0.1)  # بایاس تصادفی
neuron = Neuron(weights, bias)

# ورودی‌ها برای نورون
inputs = [0.5, 0.2, 0.8]  # سه ورودی برای نورون

# محاسبه خروجی نورون
output = neuron.forward(inputs)
print(f"خروجی نورون قبل از فعال‌سازی: {output}")

# اعمال تابع فعال‌سازی (سیگموید)
activated_output = neuron.activation(output)
print(f"خروجی نورون بعد از فعال‌سازی: {activated_output}")

# فرض می‌کنیم سیگنال خطا (دلتا) برابر 0.1 است
delta = 0.1
gradients = neuron.compute_gradient(delta)
print(f"گرادیان‌های محاسبه‌شده برای وزن‌ها: {gradients}")

# به‌روزرسانی وزن‌ها با نرخ یادگیری 0.01
neuron.update_weights(learning_rate=0.01, gradients=gradients)
print(f"وزن‌ها و بایاس بعد از به‌روزرسانی: {neuron.weights}, {neuron.bias}")

خروجی نورون قبل از فعال‌سازی: -1.077792653702492
خروجی نورون بعد از فعال‌سازی: 0
گرادیان‌های محاسبه‌شده برای وزن‌ها: [0.05, 0.020000000000000004, 0.08000000000000002]
وزن‌ها و بایاس بعد از به‌روزرسانی: [-0.9662690086364676, 0.6989679522511152, -0.9533296253659759], 0.026481960458299727


In [97]:
class Layer:
    """یه لایه توی شبکه عصبی که ورودی‌ها رو می‌گیره، می‌فرسته توی نورون‌ها، فعال‌سازی می‌کنه و خروجی می‌سازه."""

    def __init__(self, neurons: List[Neuron], is_output_layer: bool = False):
        """
        آرگومان‌ها:
          neurons: لیستی از نورون‌های این لایه.
          is_output_layer: اگه True باشه، این لایه آخره و خروجی‌شو با softmax می‌سازه.
        """
        self.neurons = neurons
        self.is_output_layer = is_output_layer

    def forward(self, inputs: List[float]) -> List[float]:
        """
        داده‌ها رو می‌فرسته توی هر نورون و خروجی خام هر نورون (logits) رو می‌گیره.
        بعد اگه لایه آخری باشه softmax می‌زنه؛ وگرنه با متد activation خود نورون سیگموید می‌زنه.

        inputs: لیست ورودی به این لایه (مثلاً خروجی لایه قبلی).
        برمی‌گردونه: لیست خروجی نهایی این لایه.
        """
        # logits یعنی «خروجی خام نورون قبل از فعال‌سازی»
        logits = [neuron.forward(inputs) for neuron in self.neurons]

        if self.is_output_layer:
            # پیاده‌سازی دستی softmax: exp هر عدد / مجموع expها
            # exp_vals = [math.exp(x) for x in logits]
            # sum_exp = sum(exp_vals)
            # return [v / sum_exp for v in exp_vals]
            return self.softmax(logits)
        else:
            # برای لایه‌های میانی: هر نورون خودش متد activation داره
            return [neuron.activation(x) for neuron, x in zip(self.neurons, logits)]

    def softmax(self, outputs: List[float]) -> List[float]:
        exps = [math.exp(x) for x in outputs]
        sum_exps = sum(exps)
        return [exp / sum_exps for exp in exps]

    def backward(self, delta: List[float], learning_rate: float) -> List[int]:
        """
        دلتا (خطا) از لایه بعدی رو می‌گیره و برای هر نورون:
        1. با compute_gradient گرادیان‌ها رو محاسبه می‌کنه
        2. با update_weights وزن و بایاس رو آپدیت می‌کنه
        3. با propagate_error_back دلتای مربوط به ورودی‌ها رو دریافت می‌کنه
        در نهایت همه دلتاها رو جمع می‌کنه و برمی‌گردونه برای لایه قبلی.

        delta: لیست خطا برای هر نورون این لایه
        learning_rate: نرخ یادگیری
        برمی‌گردونه: لیست delta برای لایه قبلی
        """
        propagated = []  # این لیست، هر عنصرش لیست خطاهایی‌یه که هر نورون برای ورودی‌ها تولید می‌کنه

        for i, neuron in enumerate(self.neurons):
            # 1. گرادیان رو خود نورون محاسبه می‌کنه
            grads = neuron.compute_gradient(delta[i])

            # 2. خود نورون وزن‌ها و بایاس رو آپدیت می‌کنه
            neuron.update_weights(learning_rate, grads)

            # 3. خطا برای ورودی‌های نورون (برای لایه قبلی)
            propagated.append(neuron.propagate_error_back())

        # جمع کردن دلتاها برای هر ورودی
        # zip(*propagated) ردیف‌ها رو ستونی می‌کنه تا بتونیم جمع کنیم
        prev_delta = [sum(x) for x in zip(*propagated)]
        return prev_delta

In [98]:
neurons = [
    Neuron(weights=[0.5, -0.4], bias=0.1),
    Neuron(weights=[-1.0, 2.0], bias=0.2)
]
layer = Layer(neurons, is_output_layer=False)

layer_output = layer.forward(inputs)
print("خروجی لایه:", layer_output)

delta = [0.1, -0.2]
prev_delta = layer.backward(delta, learning_rate=0.01)
print("دلتا برای لایه قبلی:", prev_delta)

خروجی لایه: [0.27, 0.10000000000000003]
دلتا برای لایه قبلی: [0.008541449999999978, 0.10115658000000005]


In [99]:
class Network:
    """
    یک شبکه‌ی عصبی ساده که از چند لایه تشکیل شده.
    ورودی رو می‌گیره، از توی نورون‌ها عبور می‌ده، و خروجی نهایی رو تولید می‌کنه.
    بعدش با روش backpropagation وزن‌ها رو آپدیت می‌کنه.
    """

    def __init__(self, layers: List[Layer], epochs: int, learning_rate: float):
        self.layers = layers
        self.learning_rate = learning_rate
        self.epochs = epochs

    def forward(self, inputs: List[float]) -> List[float]:
        """
        ورودی‌ها رو از طریق همه‌ی لایه‌ها عبور می‌ده و خروجی نهایی رو حساب می‌کنه.
        """
        outputs = inputs
        for layer in self.layers:
            outputs = layer.forward(outputs)
        return outputs

    def backward(self, targets: List[float], outputs: List[float]):
        """
        این تابع خطا رو از خروجی نهایی محاسبه می‌کنه و به عقب برمی‌گردونه تا وزن‌ها آپدیت بشن.
        """
        delta = self.loss_derivative(outputs, targets)
        for layer in reversed(self.layers):
            delta = layer.backward(delta, self.learning_rate)

    def compute_loss(self, predicted: List[float], actual: List[float]) -> float:
        """
        مقدار خطا رو بین خروجی پیش‌بینی‌شده و مقدار واقعی حساب می‌کنه.
        """
        return LossFunction.cross_entropy(predicted, actual)

    def loss_derivative(self, outputs: List[float], targets: List[float]) -> List[float]:
        """
        مشتق تابع خطا رو حساب می‌کنه، یعنی اختلاف بین خروجی مدل و مقدار درست.
        """
        return [pred - target for pred, target in zip(outputs, targets)]

    def train(self, training_data: List[tuple]):
        """
        شبکه رو با داده‌های آموزشی آموزش می‌ده. هر بار وزن‌ها رو آپدیت می‌کنه.
        """
        num_samples = len(training_data)
        for epoch in range(self.epochs):
            total_loss = 0
            random.shuffle(training_data)

            for inputs, targets in training_data:
                outputs = self.forward(inputs)
                loss = self.compute_loss(outputs, targets)
                total_loss += loss
                self.backward(targets, outputs)

            avg_loss = total_loss / num_samples
            print(f"Epoch {epoch + 1}/{self.epochs} complete. Average loss: {avg_loss:.4f}")

    def evaluate(self, test_data: List[tuple]) -> float:
        """
        دقت مدل رو با داده‌های تست حساب می‌کنه.
        """
        inputs_batch, targets_batch = zip(*test_data)
        predictions = [self.forward(inputs) for inputs in inputs_batch]
        accuracy = self.calculate_accuracy(predictions, targets_batch)
        return accuracy

    def predict(self, new_data: List[float]) -> List[float]:
        """
        برای یک ورودی جدید، خروجی مدل رو پیش‌بینی می‌کنه.
        """
        return self.forward(new_data)

    def calculate_accuracy(self, predictions: List[List[float]], targets: List[List[float]]) -> float:
        """
        دقت پیش‌بینی مدل رو نسبت به مقدار درست حساب می‌کنه.
        """
        correct_predictions = 0
        for pred, target in zip(predictions, targets):
            predicted_class = np.argmax(pred)
            true_class = np.argmax(target)
            if predicted_class == true_class:
                correct_predictions += 1
        return correct_predictions / len(targets)

    def save_weights(self, filename: str):
        """
        وزن‌ها و بایاس‌های نورون‌ها رو توی یه فایل ذخیره می‌کنه تا بعدا بشه دوباره بارگذاریشون کرد.
        """
        weights = [[neuron.weights for neuron in layer.neurons] for layer in self.layers]
        biases = [[neuron.bias for neuron in layer.neurons] for layer in self.layers]
        with open(filename, 'wb') as f:
            pickle.dump((weights, biases), f)

    def load_weights(self, filename: str):
        """
        وزن‌ها و بایاس‌ها رو از فایل می‌خونه و به شبکه اختصاص می‌ده.
        """
        with open(filename, 'rb') as f:
            weights, biases = pickle.load(f)
        for layer, layer_weights, layer_biases in zip(self.layers, weights, biases):
            for neuron, w, b in zip(layer.neurons, layer_weights, layer_biases):
                neuron.weights = w
                neuron.bias = b


In [100]:
class LossFunction:
    """کلاسی برای محاسبه انواع تابع هزینه."""

    @staticmethod
    def cross_entropy(predicted_outputs: List[float], actual_outputs: List[float]) -> float:
        """
        🔹 این تابع برای classification استفاده می‌شه، مخصوصاً وقتی خروجی‌ها one-hot هستن.
        🔹 از لگاریتم استفاده می‌کنه تا تفاوت بین احتمال پیش‌بینی‌شده و مقدار واقعی رو بسنجه.
        """
        predicted_outputs = np.clip(predicted_outputs, 1e-12, 1 - 1e-12)
        loss = -sum([actual * np.log(pred) for pred, actual in zip(predicted_outputs, actual_outputs)])
        return loss / len(predicted_outputs)

    @staticmethod
    def mean_squared_error(predicted_outputs: List[float], actual_outputs: List[float]) -> float:
        """
        🔸 این تابع برای regression استفاده می‌شه.
        🔸 خطای مربعی بین پیش‌بینی و مقدار واقعی رو محاسبه می‌کنه.
        """
        squared_errors = [(pred - actual) ** 2 for pred, actual in zip(predicted_outputs, actual_outputs)]
        return sum(squared_errors) / len(squared_errors)


In [101]:
class WeightsInitializer:
    """کلاسی برای مقداردهی اولیه به وزن‌ها."""

    @staticmethod
    def random_uniform(num_inputs: int) -> List[float]:
        """
        🔹 مقداردهی اولیه به وزن‌ها به صورت تصادفی بین -0.1 تا 0.1.
        🔹 برای اینکه شبکه از صفر شروع نکنه و بتونه مسیر یادگیری پیدا کنه.
        """
        return [random.uniform(-0.5, 0.5) for _ in range(num_inputs)]


# Workflow 🔮: Architecture

In [102]:
def one_hot_encode(labels: list) -> list:
    # Create a consistent label-to-index mapping
    unique_labels = sorted(set(labels))
    label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
    print("Label to Index Mapping:")
    for label, index in label_to_index.items():
        print(f"  {label}: {index}")

    # One-hot encode each label
    encoded = []
    for label in labels:
        vec = [0] * len(unique_labels)
        vec[label_to_index[label]] = 1
        encoded.append(vec)

    # Optional: show a sample
    print("\nSample One-Hot Encoded Vectors:")
    for i in range(min(3, len(encoded))):
        print(f"  {labels[i]} => {encoded[i]}")

    return encoded


## Create input layer

In [108]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Iris Classification/Iris.csv'
test_data_loading(file_path)

data, labels = load_dataset(file_path)
labels = one_hot_encode(labels)
data = min_max_normalizer(data)


dataset = list(zip(data, labels))
test_split_data()
train, val, test = split_data(dataset, training_size=0.5, validation_size=0.0)
training_data, validation_data, test_data = split_data(dataset)

print(training_data)
print(validation_data)
print(test_data)

Data loaded and validated successfully.
Label to Index Mapping:
  Iris-setosa: 0
  Iris-versicolor: 1
  Iris-virginica: 2

Sample One-Hot Encoded Vectors:
  Iris-setosa => [1, 0, 0]
  Iris-setosa => [1, 0, 0]
  Iris-setosa => [1, 0, 0]
❌ تست شکست خورد: خطا در تقسیم دیتا: تعداد نهایی با اولیه نمی‌خونه.
[([0.8456375838926175, 0.5277777777777778, 0.3333333333333332, 0.6440677966101694, 0.7083333333333334], [0, 0, 1]), ([0.4966442953020134, 0.5833333333333334, 0.3749999999999999, 0.559322033898305, 0.5], [0, 1, 0]), ([0.2080536912751678, 0.30555555555555564, 0.5833333333333333, 0.0847457627118644, 0.12500000000000003], [1, 0, 0]), ([0.8523489932885906, 0.4999999999999999, 0.41666666666666663, 0.6610169491525424, 0.7083333333333334], [0, 0, 1]), ([0.47651006711409394, 0.4999999999999999, 0.3333333333333332, 0.5084745762711864, 0.5], [0, 1, 0]), ([0.4161073825503356, 0.4722222222222222, 0.0833333333333334, 0.5084745762711864, 0.375], [0, 1, 0]), ([0.1342281879194631, 0.30555555555555564, 0.5

In [ ]:
input_layer = Layer(
    neurons=[Neuron(WeightsInitializer.random_uniform(2)) for _ in range(4)]
)

hidden_layer_1 = Layer(
    neurons=[Neuron(WeightsInitializer.random_uniform(4)) for _ in range(5)]
)

hidden_layer_2 = Layer(
    neurons=[Neuron(WeightsInitializer.random_uniform(5)) for _ in range(4)]
)

output_layer = Layer(
    neurons=[Neuron(WeightsInitializer.random_uniform(4)) for _ in range(3)],
    is_output_layer=True
)

network = Network(
    layers=[input_layer, hidden_layer_1, hidden_layer_2, output_layer],
    epochs=1000,
    learning_rate=0.05
)

network.train(train)
accuracy = network.evaluate(test)
print(f"Test Accuracy: {accuracy:.2%}")

# network.save_weights("iris_model.pkl")
# network.load_weights("iris_model.pkl")